In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import os

In [2]:
# 设定超参数
batch_size = 128
learning_rate = 0.001
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# (5000, 1, 32, 32)的numpy数组
data = np.load("data4D.npy")

In [10]:
# 将数据集划分为训练集、验证集和测试集
train_data, temp_data = train_test_split(data, test_size=0.4, random_state=42)
validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# 将NumPy数组转换为张量
train_data_tensor = torch.from_numpy(train_data)
validation_data_tensor = torch.from_numpy(validation_data)
test_data_tensor = torch.from_numpy(test_data)

# 创建DataLoader
batch_size = 128
train_dataset = TensorDataset(train_data_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

validation_dataset = TensorDataset(validation_data_tensor)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)


In [5]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [11]:
num_epochs = 100

for epoch in range(num_epochs):
    # 训练阶段
    model.train()
    train_loss = 0
    for i, (batch,) in enumerate(train_loader):
        batch = batch.to(device)
        optimizer.zero_grad()
        outputs = model(batch)
        loss = criterion(outputs, batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * len(batch)

    train_loss /= len(train_loader.dataset)

    # 验证阶段
    model.eval()
    validation_loss = 0
    with torch.no_grad():
        for i, (batch,) in enumerate(validation_loader):
            batch = batch.to(device)
            outputs = model(batch)
            loss = criterion(outputs, batch)
            validation_loss += loss.item() * len(batch)

    validation_loss /= len(validation_loader.dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {validation_loss:.4f}")

print("Training and validation complete.")

Epoch [1/100], Train Loss: 0.2745, Validation Loss: 0.0926
Epoch [2/100], Train Loss: 0.0588, Validation Loss: 0.0521
Epoch [3/100], Train Loss: 0.0524, Validation Loss: 0.0521
Epoch [4/100], Train Loss: 0.0524, Validation Loss: 0.0521
Epoch [5/100], Train Loss: 0.0524, Validation Loss: 0.0521
Epoch [6/100], Train Loss: 0.0524, Validation Loss: 0.0521
Epoch [7/100], Train Loss: 0.0524, Validation Loss: 0.0521
Epoch [8/100], Train Loss: 0.0524, Validation Loss: 0.0521
Epoch [9/100], Train Loss: 0.0524, Validation Loss: 0.0521
Epoch [10/100], Train Loss: 0.0524, Validation Loss: 0.0521
Epoch [11/100], Train Loss: 0.0524, Validation Loss: 0.0521
Epoch [12/100], Train Loss: 0.0524, Validation Loss: 0.0521
Epoch [13/100], Train Loss: 0.0524, Validation Loss: 0.0521
Epoch [14/100], Train Loss: 0.0524, Validation Loss: 0.0521
Epoch [15/100], Train Loss: 0.0524, Validation Loss: 0.0521
Epoch [16/100], Train Loss: 0.0524, Validation Loss: 0.0521
Epoch [17/100], Train Loss: 0.0524, Validation Lo

In [12]:
# 保存模型权重
model_path = "autoencoder_weights_t1.pth"
torch.save(model.state_dict(), model_path)
print("Model weights saved.")


Model weights saved.


In [13]:
# 创建测试集的DataLoader
test_dataset = TensorDataset(test_data_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 评估模型在测试集上的性能
model.eval()
test_loss = 0
with torch.no_grad():
    for i, (batch,) in enumerate(test_loader):
        batch = batch.to(device)
        outputs = model(batch)
        loss = criterion(outputs, batch)
        test_loss += loss.item() * len(batch)

test_loss /= len(test_loader.dataset)

print(f"Test Loss: {test_loss:.4f}")



Test Loss: 0.0326
